# 02-03 流与事件

理解 Node.js 的 Stream 和 EventEmitter，这是处理大数据和实时通信的基础。

## 1. EventEmitter 基础

In [ ]:
import { EventEmitter } from 'events';

// 创建事件发射器
const emitter = new EventEmitter();

// 监听事件
emitter.on('message', (data) => {
  console.log('收到消息:', data);
});

// 监听一次
emitter.once('connect', () => {
  console.log('连接成功（只触发一次）');
});

// 触发事件
emitter.emit('message', 'Hello World');
emitter.emit('connect');
emitter.emit('connect');  // 不会触发

// 移除监听器
const handler = (data) => console.log(data);
emitter.on('data', handler);
emitter.off('data', handler);  // 移除

## 2. 流（Streams）

In [ ]:
import { createReadStream, createWriteStream } from 'fs';
import { createGzip } from 'zlib';

// 可读流 - 大文件不会一次性载入内存
const readStream = createReadStream('./large-file.txt', {
  encoding: 'utf-8',
  highWaterMark: 1024  // 每次读取 1KB
});

readStream.on('data', (chunk) => {
  console.log('收到数据块:', chunk.length);
});

readStream.on('end', () => {
  console.log('读取完成');
});

// 可写流
const writeStream = createWriteStream('./output.txt');
writeStream.write('第一行\n');
writeStream.write('第二行\n');
writeStream.end();  // 结束写入

## 3. 管道（Piping）

In [ ]:
// 管道：将一个流的输出直接连接到另一个流的输入

// 复制文件（简洁写法）
createReadStream('./input.txt')
  .pipe(createWriteStream('./output.txt'));

// 压缩文件
createReadStream('./large-file.txt')
  .pipe(createGzip())
  .pipe(createWriteStream('./large-file.txt.gz'));

// 现代写法：pipeline（推荐，自动处理错误）
import { pipeline } from 'stream/promises';

await pipeline(
  createReadStream('./input.txt'),
  createGzip(),
  createWriteStream('./output.txt.gz')
);
console.log('管道处理完成');

## 4. OpenClaw 中的流处理

In [ ]:
// OpenClaw 处理语音/视频流示例
import { Transform } from 'stream';

// 自定义 Transform 流：转换数据
const jsonParser = new Transform({
  objectMode: true,
  transform(chunk, encoding, callback) {
    try {
      const parsed = JSON.parse(chunk);
      this.push(parsed);
      callback();
    } catch (err) {
      callback(err);
    }
  }
});

// 处理 WebSocket 消息流
class MessageStream extends EventEmitter {
  constructor() {
    super();
    this.buffer = '';
  }
  
  write(data) {
    this.buffer += data;
    const messages = this.buffer.split('\n');
    
    // 保留最后一个不完整的片段
    this.buffer = messages.pop();
    
    // 发送完整的消息
    messages.forEach(msg => {
      if (msg) this.emit('message', JSON.parse(msg));
    });
  }
}

const stream = new MessageStream();
stream.on('message', (msg) => console.log('收到:', msg));
stream.write('{"type":"text","content":"hi"}\n');
stream.write('{"type":"image","url":"..."}\n');

## 练习

1. 实现一个行读取器（按行读取大文件）
2. 实现一个简单的日志系统（使用 EventEmitter）
3. 用流处理实现文件拷贝，对比一次性读取的性能差异